In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'
currency_pair = 'Eur_Usd'

In [3]:
year_range = '2021-2022'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_{year_range}.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join='inner').max(axis=1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha=1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha=1/lookback).mean()

    return adx_smooth


df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,ema50,ema25,adx
0,2021-09-02 13:00:00,1.18465,1.18705,1.18458,1.18644,1.18478,1.18718,1.18471,1.18657,1.18472,1.18711,1.18464,1.18650,17685,1.183083,1.183145,1.183271,1.183525,44.561122
1,2021-09-02 17:00:00,1.18642,1.18754,1.18637,1.18744,1.18656,1.18768,1.18650,1.18756,1.18649,1.18761,1.18644,1.18750,6005,1.183380,1.183460,1.183622,1.183948,49.958819
2,2021-09-02 21:00:00,1.18747,1.18821,1.18692,1.18814,1.18761,1.18837,1.18738,1.18827,1.18754,1.18829,1.18722,1.18820,1926,1.183687,1.183786,1.183986,1.184388,52.822925
3,2021-09-03 01:00:00,1.18815,1.18836,1.18755,1.18756,1.18829,1.18850,1.18769,1.18770,1.18822,1.18842,1.18762,1.18763,4416,1.183925,1.184037,1.184264,1.184715,55.211861
4,2021-09-03 05:00:00,1.18758,1.18822,1.18706,1.18742,1.18771,1.18836,1.18720,1.18754,1.18764,1.18829,1.18714,1.18748,10407,1.184129,1.184253,1.184501,1.184987,56.611146


In [7]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,ema50,ema25,adx
1549,2022-08-30 13:00:00,1.00103,1.00410,0.99817,1.00229,1.00118,1.00423,0.99832,1.00244,1.00110,1.00416,0.99824,1.00236,37344,1.014845,1.007172,1.001718,0.999270,21.612459
1550,2022-08-30 17:00:00,1.00230,1.00308,1.00096,1.00150,1.00246,1.00323,1.00112,1.00171,1.00238,1.00316,1.00104,1.00160,15913,1.014713,1.007062,1.001714,0.999449,21.540575
1551,2022-08-30 21:00:00,1.00127,1.00339,1.00111,1.00267,1.00163,1.00355,1.00155,1.00282,1.00145,1.00346,1.00135,1.00274,5460,1.014594,1.006976,1.001754,0.999702,21.478774
1552,2022-08-31 01:00:00,1.00266,1.00442,1.00159,1.00395,1.00283,1.00455,1.00173,1.00410,1.00274,1.00448,1.00166,1.00402,10626,1.014488,1.006918,1.001843,1.000034,21.502848
1553,2022-08-31 05:00:00,1.00398,1.00455,0.99733,0.99780,1.00412,1.00468,0.99751,0.99811,1.00405,1.00462,0.99742,0.99796,22813,1.014324,1.006740,1.001690,0.999875,21.647953


In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [9]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [10]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(pips_to_risk, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close, close_half):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    win_amounts, loss_amounts, pips_risked = [], [], []
    day_fees = 0
    trade = None
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk

    if n_bars == 'any' or not wait_for_close:
        use_pullback = False

    if n_bars == 'any':
       lookback, lookforward = 12, 0

    else:
        lookback = n_bars + 1 if use_pullback else n_bars
        lookforward = -1 if use_pullback else 0

        if not wait_for_close:
            lookback, lookforward = lookback - 1, 0

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_ask_close, curr_bid_close, curr_mid_high, curr_mid_low, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Ask_Close', 'Bid_Close', 'Mid_High', 'Mid_Low', 'Date']]
        within_time_slot = start_hour <= curr_date.hour <= end_hour

        if trade is None and within_time_slot:
            spread = abs(curr_ask_open - curr_bid_open)
            buy_pips_offset, sell_pips_offset = 0, 0

            if n_bars == 'any':
                mid_opens = list(df.loc[df.index[i - lookback:i], 'Mid_Open'])
                mid_highs = list(df.loc[df.index[i - lookback:i], 'Mid_High'])
                mid_lows = list(df.loc[df.index[i - lookback:i], 'Mid_Low'])
                mid_closes = list(df.loc[df.index[i - lookback:i], 'Mid_Close'])

                def _check_bars(pips_moved, buy):
                    for j in range(len(mid_opens) - 1, -1, -1):
                        if (buy and mid_opens[j] < mid_closes[j]) or (not buy and mid_opens[j] > mid_closes[j]):
                            pips_moved += abs(mid_opens[j] - mid_closes[j])

                            if pips_moved >= pip_movement:
                                return True
                            
                        else:
                            return False
                            
                    return False
                
                def _get_pips_moved(buy):
                    moved = 0

                    for j in range(len(mid_opens) - 1, -1, -1):
                        if (buy and mid_opens[j] < mid_closes[j]) or (not buy and mid_opens[j] > mid_closes[j]):
                            moved += abs(mid_opens[j] - mid_closes[j])
                            
                        else:
                            return moved
                            
                    return moved

                if not wait_for_close:
                    buy_signal = _check_bars(abs(curr_mid_open - curr_mid_high), buy=True)
                    sell_signal = _check_bars(abs(curr_mid_open - curr_mid_low), buy=False)
                    buy_pips_offset = pip_movement - _get_pips_moved(buy=True)
                    sell_pips_offset = pip_movement - _get_pips_moved(buy=False)

                else:
                    buy_signal = _check_bars(0, buy=True)
                    sell_signal = _check_bars(0, buy=False)

            else:
                mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
                mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
                mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
                mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

                if not wait_for_close:
                    buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) + abs(curr_mid_open - curr_mid_high) >= pip_movement
                    sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) + abs(curr_mid_open - curr_mid_low) >= pip_movement
                    buy_pips_offset, sell_pips_offset = pip_movement - abs(mid_opens[0] - mid_closes[-1]), pip_movement - abs(mid_opens[0] - mid_closes[-1])

                else:
                    buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
                    sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

            if use_pullback:
                mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
                buy_signal = buy_signal and mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)
                sell_signal = sell_signal and mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

            if ema is not None:
                curr_ema, curr_mid_close = df.loc[df.index[i - 1], [ema, 'Mid_Close']]
                buy_signal = buy_signal and curr_mid_close > curr_ema
                sell_signal = sell_signal and curr_mid_close < curr_ema

            curr_adx = df.loc[df.index[i - 1], 'adx']
            moving_enough = curr_adx > adx_cutoff

            buy_signal = buy_signal and moving_enough
            sell_signal = sell_signal and moving_enough

            if invert:
                buy_signal, sell_signal = sell_signal, buy_signal

            highest_high, lowest_low = max(mid_highs), min(mid_lows)
            buy_pips_offset, sell_pips_offset = max([buy_pips_offset, 0]), max([sell_pips_offset, 0])

            if buy_signal:
                open_price = float(curr_ask_open + buy_pips_offset)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units(curr_pips_to_risk, open_price - (spread / 2), currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                 'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                 'end_date': None, 'prev_profit_ratio': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open - sell_pips_offset)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units(curr_pips_to_risk, open_price + (spread / 2), currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                'end_date': None, 'prev_profit_ratio': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and use_trailing_sl and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            if close_half and curr_profit_ratio >= 2.0 and trade['n_units'] == trade['original_units']:
                n_units_halved = int(trade['n_units'] / 2)
                trade_amount = trade['pips_risked'] * n_units_halved * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, n_units_halved)
                trade['n_units'] = n_units_halved

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'buy' and not use_trailing_sl and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and use_trailing_sl and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            if close_half and curr_profit_ratio >= 2.0 and trade['n_units'] == trade['original_units']:
                n_units_halved = int(trade['n_units'] / 2)
                trade_amount = trade['pips_risked'] * n_units_halved * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date, n_units_halved)
                trade['n_units'] = n_units_halved

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and not use_trailing_sl and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts,

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.1]
n_bars_vals = ['any']  # LEAVE THE 'any' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pip_movement_vals = [20]
use_pullback_vals = [False]
pips_to_risk_vals = ['bars', 10, 20]  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.35, 0.50]
invert_vals = [True, False]
ema_vals = [None, 'ema200', 'ema100', 'ema50', 'ema25']  # 'None' means it won't use the EMA
adx_cutoffs = [0, 25, 30]  # ADX is between 0 and 100; usually ADX > 25 means the market is moving fairly well

# Hour values are between 0 (12 AM/midnight) and 23 (11 PM).  The dates are in UTC, which is about 7 hours ahead of CA time.
# For example, if you wanted to only place trades between 6 AM and 12 PM CA time, the start hour should be 13 (1 PM UTC) and
# the end hour should be 19 (7 PM UTC).  If you want to trade all 24 hours of the day, you would set the start hour to 0 and
# the end hour to 23.
start_hours = [13, 14]
end_hours = [19, 20]
use_trailing_sl_vals = [True, False]
wait_for_close_vals = [False]
close_half_vals = [True, False]  # If True, close half the trade once it reaches a profit ratio of 2.0

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for n_bars in n_bars_vals:
            for pip_movement in pip_movement_vals:
                for use_pullback in use_pullback_vals:
                    for pips_to_risk in pips_to_risk_vals:
                        for pullback_percentage in pullback_percentages:
                            for invert in invert_vals:
                                for ema in ema_vals:
                                    for adx_cutoff in adx_cutoffs:
                                        for start_hour in start_hours:
                                            for end_hour in end_hours:
                                                for use_trailing_sl in use_trailing_sl_vals:
                                                    for wait_for_close in wait_for_close_vals:
                                                        for close_half in close_half_vals:
                                                            all_combos.append((risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close, close_half))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pips_to_risk = None
best_pullback_percentage = None
best_invert_val = None
best_ema = None
best_adx_cutoff = None
best_start_hour = None
best_end_hour = None
best_use_trailing_sl_val = None
best_wait_for_close_val = None
best_close_half_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close, close_half in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, = run_simulation(risk_reward_ratio, spread_val, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl, wait_for_close, close_half)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'ratio': risk_reward_ratio, 'spread': spread_val, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'invert': invert, 'ema': ema, 'adx_cutoff': adx_cutoff, 'start_hour': start_hour, 'end_hour': end_hour, 'use_trailing_sl': use_trailing_sl, 'wait_for_close': wait_for_close, 'close_half': close_half})


    if total_profit > best_reward:
        best_reward = total_profit
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_invert_val = invert
        best_ema = ema
        best_adx_cutoff = adx_cutoff
        best_start_hour = start_hour
        best_end_hour = end_hour
        best_use_trailing_sl_val = use_trailing_sl
        best_wait_for_close_val = wait_for_close
        best_close_half_val = close_half

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1440

-99.98195999999858 -89.14399999999999 -189.12595999999857
Num buys: 10
Num sells: 18
Num trades: 28
Num wins: 10
Num losses: 17
Win streak: 3
Loss streak: 4
Avg pips risked: 0.011321071428571408
Avg win amount: 74.98999600000005
Min win amount: 74.9512399999994
Max win amount: 75.04000000000006
Avg loss amount: -49.993054117647
Min loss amount: -49.9991199999967
Max loss amount: -49.98265999999989
Remaining runs: 1439
Best reward so far: -189.12595999999857

-99.98195999999858 -88.33986 -188.32181999999858
Num buys: 10
Num sells: 18
Num trades: 28
Num wins: 10
Num losses: 17
Win streak: 3
Loss streak: 4
Avg pips risked: 0.011321071428571408
Avg win amount: 74.98999600000005
Min win amount: 74.9512399999994
Max win amount: 75.04000000000006
Avg loss amount: -49.993054117647
Min loss amount: -49.9991199999967
Max loss amount: -49.98265999999989
Remaining runs: 1438
Best reward so far: -188.32181999999858

0 0 0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best invert val: ' + str(best_invert_val))
print('Best ema: ' + str(best_ema))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best start hour: ' + str(best_start_hour))
print('Best end hour: ' + str(best_end_hour))
print('Best use trailing sl val: ' + str(best_use_trailing_sl_val))
print('Best wait for close val: ' + str(best_wait_for_close_val))
print('Best close half val: ' + str(best_close_half_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

: 